In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:

    df = pd.read_csv("statusinvest-busca-avancada.csv", sep=';', decimal=',', thousands='.', encoding='ISO-8859-1' )
    
    df['P/L_IDEAL'] = df[' VPA'] / df[' LPA']
    df['UP/DOWNSIDE'] = (df[' VPA'] / df['PRECO'])-1
    
    valuation = df.loc[:, ['TICKER', 'PRECO', ' VPA' , 'DY', ' LPA', 'P/L', 'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 
                           'P/VP', 'EV/EBIT' ,'LIQ. CORRENTE', 'MARG. LIQUIDA', 'DIV. LIQ. / PATRI.']]
    valuation.fillna(value=0, inplace=True)
    valuation.rename(columns={'LIQ. CORRENTE': "LIQ.CORR" }, inplace=True)
    valuation.rename(columns={'MARG. LIQUIDA': "MARG.LIQ" }, inplace=True)
    valuation.rename(columns={'DIV. LIQ. / PATRI.': "DIV.LIQ/PL" }, inplace=True)
    
    
    valuation = valuation.loc[valuation['DY'] >= 10].loc[
        valuation['P/L'] >= 0.01].loc[valuation['P/L'] < 15].loc[
        valuation['ROE'] >= 10].loc[
        valuation['EV/EBIT'] >= 0.01].loc[valuation['EV/EBIT'] <= 15].loc[
        valuation['LIQ.CORR'] >= 0.5]


In [4]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

In [5]:
ticker = valuation['TICKER']
lista_stocks = ticker.to_list()

lista_indicadores_stocks = []
for stocks in lista_stocks:

    #Criando url
    url = f"https://statusinvest.com.br/acoes/{stocks}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        PRECO = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[1]/div/div[1]/strong').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[1]/strong').text
        DY = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[4]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[2]/div/span[2]/b').text
        QTDEPAPEL = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div/div[2]/div[9]/div/div/strong').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div/div[3]/div/div[3]/div/div/div/a/strong').text
        
        

        #Criando dict
        dicionario = {  "stocks": stocks, 
                        "PRECO": PRECO,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "DY": DY,
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "QTDEPAPEL": QTDEPAPEL,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        lista_indicadores_stocks.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()


In [6]:
list_stocks = pd.DataFrame.from_dict(lista_indicadores_stocks)

list_stocks = list_stocks.replace('-', '')
list_stocks = list_stocks.replace('-%', '')


list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.lstrip('R$ ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.lstrip('R$ ')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.strip('- ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.strip('- ')



list_stocks['VAL_12M'] = (list_stocks['VAL_12M']).str.rstrip('%')
list_stocks['VAL_MES'] = (list_stocks['VAL_MES']).str.rstrip('%')

list_stocks.to_csv('list_stocks.csv')



list_stocks = pd.read_csv("list_stocks.csv", sep=',', decimal=',', thousands='.', encoding='ISO-8859-1' )

list_stocks = list_stocks.drop(['Unnamed: 0'],axis=1)
list_stocks.fillna(value=0, inplace=True)


list_stocks = list_stocks.replace('AcessÃ³rios', 'Acessorios')
list_stocks = list_stocks.replace('Armas e MuniÃ§Ãµes', 'Armas e Municoes')
list_stocks = list_stocks.replace('Artefatos de Ferro e AÃ§o', 'Artefatos de Ferro e Aco')
list_stocks = list_stocks.replace('AutomÃ³veis e Motocicletas', 'Automoveis e Motocicletas')
list_stocks = list_stocks.replace('AÃ§ucar e Alcool', 'Acucar e Alcool')
list_stocks = list_stocks.replace('CalÃ§ados', 'Calcados')
list_stocks = list_stocks.replace('ConstruÃ§Ã£o Pesada', 'Construcao Pesada')
list_stocks = list_stocks.replace('EletrodomÃ©sticos', 'Eletrodomesticos')
list_stocks = list_stocks.replace('Energia ElÃ©trica', 'Energia Eletrica')
list_stocks = list_stocks.replace('Equipamentos e ServiÃ§os', 'Equipamentos e Servicos')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o de ImÃ³veis', 'Exploracao de Imoveis')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o de Rodovias', 'Exploracao de Rodovias')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o. Refino e DistribuiÃ§Ã£o', 'Exploracao Refino e Distribuicao')
list_stocks = list_stocks.replace('GestÃ£o de Recursos e Investimentos', 'Gestao Recursos e Investimentos')
list_stocks = list_stocks.replace('GÃ¡s', 'Gas')
list_stocks = list_stocks.replace('IncorporaÃ§Ãµes', 'Incorporacoes')
list_stocks = list_stocks.replace('IntermediaÃ§Ã£o ImobiliÃ¡ria', 'Intermediacao Imobiliaria')
list_stocks = list_stocks.replace('LogÃ­stica', 'Logistica')
list_stocks = list_stocks.replace('Material AeronÃ¡utico e de Defesa', 'Material Aeronautico e Defesa')
list_stocks = list_stocks.replace('Material RodoviÃ¡rio', 'Material Rodoviario')
list_stocks = list_stocks.replace('Minerais MetÃ¡licos', 'Minerais Metalicos')
list_stocks = list_stocks.replace('MÃ¡q. e Equip. ConstruÃ§Ã£o e AgrÃ­colas', 'Maq Equip Construcao e Agricolas')
list_stocks = list_stocks.replace('MÃ¡q. e Equip. Industriais', 'Maq Equip Industriais')
list_stocks = list_stocks.replace('MÃ³veis', 'Moveis')
list_stocks = list_stocks.replace('PetroquÃ­micos', 'Petroquimicos')
list_stocks = list_stocks.replace('Produtos para ConstruÃ§Ã£o', 'Produtos para Construcao')
list_stocks = list_stocks.replace('ProduÃ§Ã£o de Eventos e Shows', 'Producao de Eventos e Shows')
list_stocks = list_stocks.replace('ProduÃ§Ã£o e DifusÃ£o de Filmes e Programas', 'Producao Difusao de Filmes e Programas')
list_stocks = list_stocks.replace('Programas de FidelizaÃ§Ã£o', 'Programas de Fidelizacao')
list_stocks = list_stocks.replace('Programas e ServiÃ§os', 'Programas e Servicos')
list_stocks = list_stocks.replace('QuÃ­micos Diversos', 'Quimicos Diversos')
list_stocks = list_stocks.replace('Serv.MÃ©d.Hospit..AnÃ¡lises e DiagnÃ³sticos', 'Serv Mao Hosp Analises e Diagnosticos')
list_stocks = list_stocks.replace('ServiÃ§os Diversos', 'Servicos Diversos')
list_stocks = list_stocks.replace('ServiÃ§os Educacionais', 'Servicos Educacionais')
list_stocks = list_stocks.replace('ServiÃ§os Financeiros Diversos', 'Servicos Financeiros Diversos')
list_stocks = list_stocks.replace('Soc. CrÃ©dito e Financiamento', 'Soc Credito e Financiamento')
list_stocks = list_stocks.replace('Tecidos. VestuÃ¡rio e CalÃ§ados', 'Tecidos Vestuario e Calcados')
list_stocks = list_stocks.replace('TelecomunicaÃ§Ãµes', 'Telecomunicacoes')
list_stocks = list_stocks.replace('Transporte AÃ©reo', 'Transporte Aereo')
list_stocks = list_stocks.replace('Transporte FerroviÃ¡rio', 'Transporte Ferroviario')
list_stocks = list_stocks.replace('Transporte HidroviÃ¡rio', 'Transporte Hidroviario')
list_stocks = list_stocks.replace('UtensÃ­lios DomÃ©sticos', 'Utensilios Domesticos')
list_stocks = list_stocks.replace('VestuÃ¡rio', 'Vestuario')
list_stocks = list_stocks.replace('Ãgua e Saneamento', 'Agua e Saneamento')


In [7]:
valuation.rename(columns={'TICKER': "TICKERS" }, inplace=True)
valuation

,TICKERS,PRECO,VPA,DY,LPA,P/L,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL
7,AGRO3,28.90,21.46,12.49,2.62,9.88,8.190840,-0.257439,12.22,1.21,11.44,2.14,20.98,0.21
36,AURE3,13.93,14.79,10.90,3.10,4.44,4.770968,0.061737,20.94,0.93,8.88,2.12,52.70,0.03
51,BBSE3,31.94,4.47,11.21,3.59,8.66,1.245125,-0.860050,80.32,6.95,7.54,0.98,0.00,0.00
61,BGIP4,23.50,35.35,10.66,3.65,6.35,9.684932,0.504255,10.32,0.65,6.18,0.76,5.27,0.00
87,BRAP3,20.88,19.30,12.29,7.01,3.09,2.753210,-0.075670,36.31,1.12,3.11,4.70,0.00,-0.03
88,BRAP4,22.60,19.30,12.58,7.01,3.32,2.753210,-0.146018,36.31,1.21,3.11,4.70,0.00,-0.03
125,CATA3,103.61,1214.17,38.09,211.94,0.49,5.728838,10.718657,17.46,0.09,1.91,1.85,9.25,0.27
132,CEBR3,13.62,13.92,10.97,2.61,5.20,5.333333,0.022026,18.78,0.98,1.55,5.72,66.35,-0.62
133,CEBR5,13.70,13.92,11.03,2.61,5.17,5.333333,0.016058,18.78,0.97,1.55,5.72,66.35,-0.62
134,CEBR6,15.15,13.92,11.16,2.61,5.62,5.333333,-0.081188,18.78,1.06,1.55,5.72,66.35,-0.62


In [8]:
list_stocks.rename(columns={'stocks': "TICKERS" }, inplace=True)
list_stocks

,TICKERS,PRECO,MIN_MES,MAX_MES,MIN_52S,MAX_52S,DY,VAL_12M,VAL_MES,QTDEPAPEL,SEGMENTO
0,AGRO3,28.90,24.06,28.90,22.59,31.35,12.49,6.06,18.25,102377008,Agricultura
1,AURE3,13.93,13.77,13.93,12.10,14.95,10.90,7.57,0.14,1000000000,Energia Eletrica
2,BBSE3,31.94,31.03,31.94,23.95,34.18,11.21,25.95,2.93,2000000000,Seguradoras
3,BGIP4,23.50,22.12,23.50,15.08,27.39,10.66,42.34,6.24,16905980,Bancos
4,BRAP3,20.88,20.80,21.97,18.59,27.04,12.29,12.32,-4.48,393096610,Minerais Metalicos
5,BRAP4,22.60,22.56,23.95,20.18,31.20,12.58,11.66,-4.36,393096610,Minerais Metalicos
6,CATA3,103.61,103.61,113.46,0.00,0.00,38.09,0.00,0.00,145307,Fios e Tecidos
7,CEBR3,13.62,13.59,13.62,8.64,13.78,10.97,19.16,0.15,72081915,Energia Eletrica
8,CEBR5,13.70,13.36,13.70,9.82,14.00,11.03,34.18,0.74,72081915,Energia Eletrica
9,CEBR6,15.15,14.66,15.15,9.66,15.15,11.16,35.39,3.34,72081915,Energia Eletrica


In [9]:
Investing = pd.merge(valuation, list_stocks, on='TICKERS')
Investing

,TICKERS,PRECO_x,VPA,DY_x,LPA,P/L,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,...,PRECO_y,MIN_MES,MAX_MES,MIN_52S,MAX_52S,DY_y,VAL_12M,VAL_MES,QTDEPAPEL,SEGMENTO
0,AGRO3,28.90,21.46,12.49,2.62,9.88,8.190840,-0.257439,12.22,1.21,...,28.90,24.06,28.90,22.59,31.35,12.49,6.06,18.25,102377008,Agricultura
1,AURE3,13.93,14.79,10.90,3.10,4.44,4.770968,0.061737,20.94,0.93,...,13.93,13.77,13.93,12.10,14.95,10.90,7.57,0.14,1000000000,Energia Eletrica
2,BBSE3,31.94,4.47,11.21,3.59,8.66,1.245125,-0.860050,80.32,6.95,...,31.94,31.03,31.94,23.95,34.18,11.21,25.95,2.93,2000000000,Seguradoras
3,BGIP4,23.50,35.35,10.66,3.65,6.35,9.684932,0.504255,10.32,0.65,...,23.50,22.12,23.50,15.08,27.39,10.66,42.34,6.24,16905980,Bancos
4,BRAP3,20.88,19.30,12.29,7.01,3.09,2.753210,-0.075670,36.31,1.12,...,20.88,20.80,21.97,18.59,27.04,12.29,12.32,-4.48,393096610,Minerais Metalicos
5,BRAP4,22.60,19.30,12.58,7.01,3.32,2.753210,-0.146018,36.31,1.21,...,22.60,22.56,23.95,20.18,31.20,12.58,11.66,-4.36,393096610,Minerais Metalicos
6,CATA3,103.61,1214.17,38.09,211.94,0.49,5.728838,10.718657,17.46,0.09,...,103.61,103.61,113.46,0.00,0.00,38.09,0.00,0.00,145307,Fios e Tecidos
7,CEBR3,13.62,13.92,10.97,2.61,5.20,5.333333,0.022026,18.78,0.98,...,13.62,13.59,13.62,8.64,13.78,10.97,19.16,0.15,72081915,Energia Eletrica
8,CEBR5,13.70,13.92,11.03,2.61,5.17,5.333333,0.016058,18.78,0.97,...,13.70,13.36,13.70,9.82,14.00,11.03,34.18,0.74,72081915,Energia Eletrica
9,CEBR6,15.15,13.92,11.16,2.61,5.62,5.333333,-0.081188,18.78,1.06,...,15.15,14.66,15.15,9.66,15.15,11.16,35.39,3.34,72081915,Energia Eletrica
